In [1]:
import config
import datetime
# import itertools
import os
import pandas as pd
import requests
import tweepy as tw
import threading
import time
import queue
from pathlib import Path

In [2]:
def get_df():
    return pd.DataFrame(
        columns=[
            "tweet_id",
            "name",
            "screen_name",
            "retweet_count",
            "text",
            "mined_at",
            "created_at",
            "favourite_count",
            "hashtags",
            "status_count",
            "followers_count",
            "location",
            "source_device",
        ]
    )

In [3]:
#### Twitter API ####
class TweetMiner(object):
    result_minit = 20
    ret = []
    api = False

    twitter_keys = {
        "consumer_key": config.consumer_key,
        "consumer_secret": config.consumer_secret,
        "access_token_key": config.access_token_key,
        "access_token_secret": config.access_token_secret,
    }

    def __init__(self, keys_dict=twitter_keys, api=api):
        """ 
        Initialize the miner.
        """
        self.api = api
        auth = tw.OAuthHandler(keys_dict["consumer_key"], keys_dict["consumer_secret"])
        auth.set_access_token(keys_dict["access_token_key"], keys_dict["access_token_secret"])
        self.api = tw.API(auth, wait_on_rate_limit=True)
        self.twitter_keys = keys_dict
        self.path = Path(f'{os.getcwd()}')

    def mine_tweets(self, query="BTC"):
        """
        Mine tweets from the query.
        """
        last_tweet_id = False
        page_num = 1

        ret = get_df()
        crypto_query = f"#{query}" # since:2022-01-24 until:2022-01-26"
        print("========", query, crypto_query)

        document_count = 0
        for page in tw.Cursor(
            self.api.search_tweets, 
            q=crypto_query, 
            lang="en", 
            tweet_mode="extended", 
            count=200
        ).pages():
            print("........... new page", page_num)
            page_num += 1

            for tweet in page:
                data = {
                    "tweet_id": tweet.id,
                    "name": tweet.user.name,
                    "screen_name": tweet.user.screen_name,
                    "retweet_count": tweet.retweet_count,
                    "text": tweet.full_text,
                    "mined_at": datetime.datetime.now(),
                    "created_at": tweet.created_at,
                    "favourite_count": tweet.favorite_count,
                    "hashtags": tweet.entities.get("hashtags"),
                    "status_count": tweet.user.statuses_count,
                    "followers_count": tweet.user.followers_count,
                    "location": tweet.user.location,
                    "source_device": tweet.source,
                }
                try:
                    data["retweet_text"] = tweet.retweeted_status.full_text
                except:
                    data["retweet_text"] = "None"

                last_tweet_id = tweet.id
                ret = ret.append(data, ignore_index=True)

            
            if page_num % 180 == 0:
                date_label = datetime.datetime.now().strftime("%Y-%m-%d")
                file_name = f"{date_label}-{query}-{document_count}.csv"
                print("Saving to file:", file_name, " after page number ", page_num)
                ret.to_csv(file_name, index=False)
                document_count += 1
                print("Resetting df")
                ret = get_df()
        date_label = datetime.datetime.now().strftime("%Y-%m-%d")
        ret.to_csv(f"{self.Path}\{date_label}-{query}-{page_num}.csv", index=False)

In [4]:
miner = TweetMiner()

handle_list = [
    #"DOGE",
    "BTC",
    #"ETH",
    #"USDT",
    #"XRP",
    #"BCH",
    #"ADA",
    #"BSV",
    #"LTC",
    #"LINK",
    #"BNB",
    #"EOS",
    #"TRON",
]

In [5]:
should_publish = threading.Event()
update_queue = queue.Queue()


def start_publisher():
    """ 
    Start the publisher thread. (This is the thread that will be listening for updates)
    """
    global handle_list

    starttime = time.time()
    print("Start polling", starttime)
    poll_iteration = 1

    for i in range(10):
        for name in handle_list[:1]:
            print(i, poll_iteration, "\rpublishing update ", end="")
            update_queue.put((poll_iteration, name))
            poll_iteration += 1
            time.sleep(900)
            print("\rawaiting for publishing update", end="")
            should_publish.wait()
            update_queue.join()

def start_update_listener():
    """
    Start the update listener thread. (This is the thread that will be listening for updates)
    """
    while True:
        poll_iteration, name = update_queue.get()

        print(" --- ", name)
        try:

            miner.mine_tweets(query=name)
            update_queue.task_done()

        except Exception as e:  # work on python 3.x
            print("Failed to upload to ftp: " + str(e))

# Start the threads
listener_thread = threading.Thread(target=start_update_listener, daemon=True)
publisher_thread = threading.Thread(target=start_publisher, daemon=True)

In [6]:
publisher_thread.start()
listener_thread.start()
# start publishing
should_publish.set()

Start polling 1646331807.1295683
publishing update  ---  BTC
======== BTC #BTC


Rate limit reached. Sleeping for: 264


........... new page 1
........... new page 2
........... new page 3
........... new page 4
........... new page 5
........... new page 6
........... new page 7
........... new page 8
........... new page 9
........... new page 10
........... new page 11
........... new page 12
........... new page 13
........... new page 14
........... new page 15
........... new page 16
........... new page 17
........... new page 18
........... new page 19
........... new page 20
........... new page 21
........... new page 22
........... new page 23
........... new page 24
........... new page 25
........... new page 26
........... new page 27
........... new page 28
........... new page 29
........... new page 30
........... new page 31
........... new page 32
........... new page 33
........... new page 34
........... new page 35
........... new page 36
........... new page 37
........... new page 38
........... new page 39
........... new page 40
........... new page 41
........... new page 42
.

Rate limit reached. Sleeping for: 689


awaiting for publishing update........... new page 181
........... new page 182
........... new page 183
........... new page 184
........... new page 185
........... new page 186
........... new page 187
........... new page 188
........... new page 189
........... new page 190
........... new page 191
........... new page 192
........... new page 193
........... new page 194
........... new page 195
........... new page 196
........... new page 197
........... new page 198
........... new page 199
........... new page 200
........... new page 201
........... new page 202
........... new page 203
........... new page 204
........... new page 205
........... new page 206
........... new page 207
........... new page 208
........... new page 209
........... new page 210
........... new page 211
........... new page 212
........... new page 213
........... new page 214
........... new page 215
........... new page 216
........... new page 217
........... new page 218
........... new page

...

........... new page 2521
........... new page 2522
........... new page 2523
........... new page 2524
........... new page 2525
........... new page 2526
........... new page 2527
........... new page 2528
........... new page 2529
........... new page 2530
........... new page 2531
........... new page 2532
........... new page 2533
........... new page 2534
........... new page 2535
........... new page 2536
........... new page 2537
........... new page 2538
........... new page 2539
........... new page 2540
........... new page 2541
........... new page 2542
........... new page 2543
........... new page 2544
........... new page 2545
........... new page 2546
........... new page 2547
........... new page 2548
........... new page 2549
........... new page 2550
........... new page 2551
........... new page 2552
........... new page 2553
........... new page 2554
........... new page 2555
........... new page 2556
........... new page 2557
........... new page 2558
........... 

Rate limit reached. Sleeping for: 646


........... new page 2701
........... new page 2702
........... new page 2703
........... new page 2704
........... new page 2705
........... new page 2706
........... new page 2707
........... new page 2708
........... new page 2709
........... new page 2710
........... new page 2711
........... new page 2712
........... new page 2713
........... new page 2714
........... new page 2715
........... new page 2716
........... new page 2717
........... new page 2718
........... new page 2719
........... new page 2720
........... new page 2721
........... new page 2722
........... new page 2723
........... new page 2724
........... new page 2725
........... new page 2726
........... new page 2727
........... new page 2728
........... new page 2729
........... new page 2730
........... new page 2731
........... new page 2732
........... new page 2733
........... new page 2734
........... new page 2735
........... new page 2736
........... new page 2737
........... new page 2738
........... 

Rate limit reached. Sleeping for: 676


........... new page 2881
........... new page 2882
........... new page 2883
........... new page 2884
........... new page 2885


In [7]:
should_publish.clear()

........... new page 2886
........... new page 2887
........... new page 2888
........... new page 2889
........... new page 2890
........... new page 2891
........... new page 2892
........... new page 2893
........... new page 2894
........... new page 2895
........... new page 2896
........... new page 2897
........... new page 2898
........... new page 2899
........... new page 2900
........... new page 2901
........... new page 2902
........... new page 2903
........... new page 2904
........... new page 2905
........... new page 2906
........... new page 2907
........... new page 2908
........... new page 2909
........... new page 2910
........... new page 2911
........... new page 2912
........... new page 2913
........... new page 2914
........... new page 2915
........... new page 2916
........... new page 2917
........... new page 2918
........... new page 2919
........... new page 2920
........... new page 2921
........... new page 2922
........... new page 2923
........... 

Rate limit reached. Sleeping for: 277


........... new page 3061
........... new page 3062
........... new page 3063
........... new page 3064
........... new page 3065
........... new page 3066
........... new page 3067
........... new page 3068
........... new page 3069
........... new page 3070
........... new page 3071
........... new page 3072
........... new page 3073
........... new page 3074
........... new page 3075
........... new page 3076
........... new page 3077
........... new page 3078
........... new page 3079
........... new page 3080
........... new page 3081
........... new page 3082
........... new page 3083
........... new page 3084
........... new page 3085
........... new page 3086
........... new page 3087
........... new page 3088
........... new page 3089
........... new page 3090
........... new page 3091
........... new page 3092
........... new page 3093
........... new page 3094
........... new page 3095
........... new page 3096
........... new page 3097
........... new page 3098
........... 

Rate limit reached. Sleeping for: 669


........... new page 3241
........... new page 3242
........... new page 3243
........... new page 3244
........... new page 3245
........... new page 3246
........... new page 3247
........... new page 3248
........... new page 3249
........... new page 3250
........... new page 3251
........... new page 3252
........... new page 3253
........... new page 3254
........... new page 3255
........... new page 3256
........... new page 3257
........... new page 3258
........... new page 3259
........... new page 3260
........... new page 3261
........... new page 3262
........... new page 3263
........... new page 3264
........... new page 3265
........... new page 3266
........... new page 3267
........... new page 3268
........... new page 3269
........... new page 3270
........... new page 3271
........... new page 3272
........... new page 3273
........... new page 3274
........... new page 3275
........... new page 3276
........... new page 3277
........... new page 3278
........... 

Rate limit reached. Sleeping for: 626


........... new page 3421
........... new page 3422
........... new page 3423
........... new page 3424
........... new page 3425
........... new page 3426
........... new page 3427
........... new page 3428
........... new page 3429
........... new page 3430
........... new page 3431
........... new page 3432
........... new page 3433
........... new page 3434
........... new page 3435
........... new page 3436
........... new page 3437
........... new page 3438
........... new page 3439
........... new page 3440
........... new page 3441
........... new page 3442
........... new page 3443
........... new page 3444
........... new page 3445
........... new page 3446
........... new page 3447
........... new page 3448
........... new page 3449
........... new page 3450
........... new page 3451
........... new page 3452
........... new page 3453
........... new page 3454
........... new page 3455
........... new page 3456
........... new page 3457
........... new page 3458
........... 

Rate limit reached. Sleeping for: 678


........... new page 3601
........... new page 3602
........... new page 3603
........... new page 3604
........... new page 3605
........... new page 3606
........... new page 3607
........... new page 3608
........... new page 3609
........... new page 3610
........... new page 3611
........... new page 3612
........... new page 3613
........... new page 3614
........... new page 3615
........... new page 3616
........... new page 3617
........... new page 3618
........... new page 3619
........... new page 3620
........... new page 3621
........... new page 3622
........... new page 3623
........... new page 3624
........... new page 3625
........... new page 3626
........... new page 3627
........... new page 3628
........... new page 3629
........... new page 3630
........... new page 3631
........... new page 3632
........... new page 3633
........... new page 3634
........... new page 3635
........... new page 3636
........... new page 3637
........... new page 3638
........... 

Rate limit reached. Sleeping for: 669
